In [1]:
# Cell 1: Imports
import numpy as np
import torch
import matplotlib.pyplot as plt
from env import TreasureGuardianEnv
from maddpg import MADDPG

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


C:\Users\malli\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Using device: cpu


In [2]:
env = TreasureGuardianEnv()
print("Observation Space:", env.observation_space)
print("Observation Space Type:", type(env.observation_space))


Observation Space: Dict('guardian': Box(0, 9, (2,), int32), 'keys': Box(0, 9, (3, 2), int32), 'pits': Box(0, 9, (2, 2), int32), 'treasure': Box(0, 9, (2,), int32), 'villains': Box(0, 9, (1, 2), int32), 'walls': Box(0, 9, (15, 2), int32))
Observation Space Type: <class 'gymnasium.spaces.dict.Dict'>


In [3]:
env = TreasureGuardianEnv()
num_agents = 1 + env.num_villains  # Guardian + Villains
maddpg = MADDPG(env=env, num_agents=num_agents)
print("MADDPG initialized successfully!")


MADDPG initialized successfully!


In [4]:
print("Observation Space Keys:", env.observation_space.spaces.keys())


Observation Space Keys: dict_keys(['guardian', 'keys', 'pits', 'treasure', 'villains', 'walls'])


In [5]:
n_episodes = 3000
max_steps = 100
reward_log = []

for ep in range(1, n_episodes + 1):
    obs_raw, _ = env.reset()
    # Construct a list of observations: first the guardian, then villains.
    obs = [obs_raw["guardian"]] + [v for v in obs_raw["villains"]]
    total_reward = np.zeros(num_agents)

    for step in range(max_steps):
        # Assume maddpg.act() returns a list of actions, one per agent.
        action_list = maddpg.act(obs)
        # Convert the flat list into dictionary format for the environment:
        actions = {
            "guardian": action_list[0],
            "villains": action_list[1:]
        }
        
        # Step in the environment. Your env.step() returns:
        # (next_obs, (guardian_reward, villain_reward), done, info)
        next_obs_raw, rewards_raw, done_raw, _ = env.step(actions)
        # Convert next_obs: list format with guardian first.
        next_obs = [next_obs_raw["guardian"]] + [v for v in next_obs_raw["villains"]]
        
        # Since rewards_raw is a tuple (guardian_reward, villain_reward),
        # we convert it to a list with guardian and villain rewards.
        rewards = [rewards_raw[0], rewards_raw[1]]
        
        # If done_raw is a single boolean flag, we create a list for each agent.
        # (If your env returns a tuple/dict, adjust accordingly.)
        done = [done_raw, done_raw]
        
        # Perform a training step with MADDPG
        maddpg.step(obs, action_list, rewards, next_obs, done)

        obs = next_obs
        total_reward += np.array(rewards)

        if any(done):
            break

    reward_log.append(total_reward)

    if ep % 100 == 0:
        avg_rewards = np.mean(reward_log[-100:], axis=0)
        print(f"Episode {ep} - Avg Reward: {avg_rewards}")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1024,) + inhomogeneous part.

In [ ]:
# Cell 4: Plot Rewards

reward_log = np.array(reward_log)
plt.figure(figsize=(12, 6))
for i in range(num_agents):
    plt.plot(reward_log[:, i], label=f"Agent {i}")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Training Rewards per Agent")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Cell 5: Save Trained Models
maddpg.save("maddpg_models/")
print("Models saved to 'maddpg_models/'")
